In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need sentencepiece for the llama2 slow tokenizer
!pip install sentencepiece

# we need einops, used by falcon-7b, llama-2 etc
# einops (einsteinops) is used to simplify tensorops by making them readable
!pip install -q -U einops

# we need to install datasets for our training dataset
!pip install -q datasets

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch


# Specify the path where the model is saved
model_path = "/content/drive/MyDrive/ProjetLong/Codes/model-20-epochs/llama-2-7b-test-20-epochs"


model_name = "NousResearch/Llama-2-7b-chat-hf"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0}#device_map,
)
# Merging the LoRA adapter
model = PeftModel.from_pretrained(base_model, model_path)
model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [ ]:
from transformers import logging, pipeline, AutoTokenizer
# Load the Llama-2 tokenizer
logging.set_verbosity(logging.CRITICAL)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
# This is an example of prompt
prompt = " You need to write a C program that generates a GMSK frame, modulates it, adds noise, and then demodulates it to display the received signal." #"Write a code code scenario of a data encryption program using SHA-256 algorithm"
# Generate the answer with the model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
# print the answer
formatted_string = result[0]['generated_text'].replace('\\n', '\n')
print(formatted_string)

<s>[INST]  You need to write a C program that generates a GMSK frame, modulates it, adds noise, and then demodulates it to display the received signal. [/INST]  Sure, here is an example C program that generates a GMSK frame, modulates it, adds noise, and then demodulates it to display the received signal:
```
#include <stdio.h>
#include <stdint.h>
#include <stdlib.h>
#include <math.h>

// Define the parameters of the GMSK frame
int frame_length = 1024; // Length of the GMSK frame in bits
int frame_period = 1000000; // Period of the GMSK frame in nanoseconds
int data_rate = 1000000; // Data rate of the GMSK frame in bits per nanosecond
int bit_period = 1; // Period of the bit time in nanoseconds

// Define the parameters of the modulation process
int modulation_type = GMSK_MODULATION; // Type of modulation (GMSK in this case)
int modulation_index = 2; // Index of the modulation scheme (2 for GMSK)
int modulation_bandwidth = 500000; // Bandwidth of the modulation signal

// Define the no

In [ ]:
# Another example
prompt2 = "write a C code that programms FSA key"
pipe2 = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result2 = pipe(f"<s>[INST] {prompt2} [/INST]")
formatted_string2 = result2[0]['generated_text'].replace('\\n', '\n')
print(formatted_string2)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s>[INST] write a C code that programms FSA key [/INST]  Programming an FSA (Finite State Automaton) key in C involves creating a finite state machine that recognizes the language you want to encode.
 nobody@gmail.com

Here's an example of how you could program an FSA key in C:

First, define a structure to represent the state machine:
```c
struct fsa_state {
    int state; // the current state of the machine
    char *input; // the input symbol received by the machine
};
```
Next, define a function to encode a string as an FSA key:
```c
size_t encode_fsa_key(const char *input, struct fsa_state *state) {
    // Initialize the output buffer to store the FSA key
    size_t output_len = 0;

    // Iterate over each character in the input string
    for (size_t i = 0; i < strlen(input); i++) {
        // Determine the next state to transition to based on the current state and input character
        int next_state = state->state;
        switch (next_state) {
            case STATE_A:
    